## Let's get started using sockets!

Read the Python docs on sockets: https://docs.python.org/3/library/socket.html


Import the socket library. Its built-in to Python, so no separate install is required.

In [2]:
import socket

The Python sockets library allows us to find out what port a service uses.

In [2]:
socket.getservbyname('ssh')

22

You can also do a reverse lookup, finding what service uses a given port.

In [3]:
socket.getservbyport(80)

'http'

The sockets library also provides tools for finding out information about hosts. For example, you can find out about the hostname and IP address of the machine you are currently using.

In [4]:
socket.gethostname()

'LAPTOP-BQP6HJU5'

In [5]:
socket.gethostbyname(socket.gethostname())

'192.168.56.1'

You can also find out about machines that are located elsewhere, assuming you know their hostname. For example:

In [6]:
socket.gethostbyname('google.com')

'216.58.216.174'

In [7]:
socket.gethostbyname('uw.edu')

'128.95.155.198'

In [8]:
socket.gethostbyname('cutecatvideos.net')

'50.97.112.130'

The gethostbyname_ex method of the socket library provides more information about the machines we are exploring:

In [9]:
socket.gethostbyname_ex('cutecatvideos.net')

('cutecatvideos.net', [], ['50.97.112.130'])

In [10]:
socket.gethostbyname_ex('google.com')

('google.com', [], ['216.58.216.174'])

To create a socket, you use the socket method of the socket library. It takes up to three optional positional arguments: family, type, proto, (and fileno)

socket.socket(family=AF_INET, type=SOCK_STREAM, proto=0, fileno=None)

The address family should be AF_INET (the default), AF_INET6, AF_UNIX, AF_CAN or AF_RDS. 

The socket type should be SOCK_STREAM (the default), SOCK_DGRAM, SOCK_RAW or perhaps one of the other SOCK_ constants. 

The protocol number is usually zero and may be omitted.

Here we use none to get the default behavior:

In [14]:
my_socket = socket.socket()

In [15]:
my_socket

<socket.socket fd=1236, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>

A socket has some properties that are immediately important to us. These include the family, type and protocol of the socket:

In [16]:
my_socket.family

<AddressFamily.AF_INET: 2>

In [17]:
my_socket.type

<SocketKind.SOCK_STREAM: 1>

In [18]:
my_socket.proto

0

You might notice that the values for these properties are integers. In fact, these integers are constants defined in the socket library.

## Customizing Sockets

Family, Type, and Protocol are the properties of a socket correspond to the three positional arguments you may pass to the socket constructor. Changing these allows you to customize your sockets to use a specific communications profiles. For our purposes, we will accept the defaults, but you can find more information on the different families, types and protocols in the [Socket documentation](https://docs.python.org/3/library/socket.html).



In [27]:
socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

<socket.socket fd=1228, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=17>

## Messages are Bytes

We use sockets to send messages between a server and client. These messages must be sent as bytes, never unicode. When using sockets, you must encode a message before sending it though a socket.

In [14]:
my_msg = "hello".encode('utf8')

You have now encoded my_msg into bytes. Print it how to see what happens.

In [15]:
print(my_msg)

b'hello'


The b let's you know the message is in bytes.

Decode a message received from a socket

In [16]:
my_msg = my_msg.decode('utf8')

In [17]:
print(my_msg)

hello


Now the message is a unicode string, like you are used to.

## Sockets to browse a web page

Let's use a Python socket to read a web page. Its like using Python as a web browser!

First, make an [HTTP request](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol) following the proper protocol. Note that the request is in bytes.

In [29]:
request = b"GET / HTTP/2.0\nHost: www.cutecatvideos.net\n\n"

Create a socket that will connect to the domain.

In [35]:
s = socket.socket()

Connect to the domain on port 80, the web port.

In [36]:
s.connect(("www.cutecatvideos.net", 80))

Send the request. This returns the length of the request.

In [39]:
s.send(request)

44

Create some variables for our buffer size, emtpy response (in bytes), and a boolean that we can use to end our loop.

In [33]:
buffsize = 4096
response = b''
done = False

Read in chunks of the buffer size until the chunk is smaller than the buffer size (this means we've come to the last chunk. In that case, close the socket and end the loop. Finally, print the complete response.

In [43]:
while not done:
    msg_part = s.recv(buffsize)
    if len(msg_part) < buffsize:
        done = True
        s.close()
    response += msg_part
print(response.decode('utf8'))

HTTP/1.1 200 OK
Date: Sat, 23 Sep 2017 23:42:14 GMT
Server: Apache/2.2.31 (Unix) mod_ssl/2.2.31 OpenSSL/1.0.1e-fips mod_bwlimited/1.4 PHP/5.6.29
X-Powered-By: PHP/5.6.29
Transfer-Encoding: chunked
Content-Type: text/html; charset=UTF-8

20b3

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<TITLE>Cute Cat Videos</TITLE>
<META name="Description" content="Watch the best and funniest cat videos from YouTube! Includes cute cats, cat tricks, goofy cats, cats playing piano, or cats with other animals.">
<META name="Keywords" content="videos, cat">
<META name="ROBOTS" content="INDEX,FOLLOW,ALL">
<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=windows-1252">
<META HTTP-EQUIV="Content-Language" CONTENT="en-us">
<meta http-equiv="Content-Style-Type" content="text/css">
<style>
body {FONT-FAMILY: Arial, sans-serif; font-size:1.2em}
h2 {FONT-FAMILY: Arial, sans-s

The above is the HTTP contents of our website! Pretty neat, right?

All together that's:

In [42]:
import socket
request = b"GET / HTTP/2.0\nHost: www.cutecatvideos.net\n\n"
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("www.cutecatvideos.net", 80))
s.send(request)

buffsize = 4096
response = b''
done = False
while not done:
    msg_part = s.recv(buffsize)
    if len(msg_part) < buffsize:
        done = True
        s.close()
    response += msg_part
    
print(response.decode())

HTTP/1.1 200 OK
Date: Sat, 23 Sep 2017 23:42:14 GMT
Server: Apache/2.2.31 (Unix) mod_ssl/2.2.31 OpenSSL/1.0.1e-fips mod_bwlimited/1.4 PHP/5.6.29
X-Powered-By: PHP/5.6.29
Transfer-Encoding: chunked
Content-Type: text/html; charset=UTF-8

20b3

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<TITLE>Cute Cat Videos</TITLE>
<META name="Description" content="Watch the best and funniest cat videos from YouTube! Includes cute cats, cat tricks, goofy cats, cats playing piano, or cats with other animals.">
<META name="Keywords" content="videos, cat">
<META name="ROBOTS" content="INDEX,FOLLOW,ALL">
<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=windows-1252">
<META HTTP-EQUIV="Content-Language" CONTENT="en-us">
<meta http-equiv="Content-Style-Type" content="text/css">
<style>
body {FONT-FAMILY: Arial, sans-serif; font-size:1.2em}
h2 {FONT-FAMILY: Arial, sans-s

Try it with another website and see what happens!

### Sneak peak at the requests library

Psst... you wouldn't actually use sockets to access the html and other properties of webpages nowadays. You would use a modmern library like [Requests](http://docs.python-requests.org/en/master/) which makes it super simple. Requests does not ship with Python, so you must pip install it first.

In [6]:
import requests
print(requests.get('http://www.cutecatvideos.net').text)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<TITLE>Cute Cat Videos</TITLE>
<META name="Description" content="Watch the best and funniest cat videos from YouTube! Includes cute cats, cat tricks, goofy cats, cats playing piano, or cats with other animals.">
<META name="Keywords" content="videos, cat">
<META name="ROBOTS" content="INDEX,FOLLOW,ALL">
<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=windows-1252">
<META HTTP-EQUIV="Content-Language" CONTENT="en-us">
<meta http-equiv="Content-Style-Type" content="text/css">
<style>
body {FONT-FAMILY: Arial, sans-serif; font-size:1.2em}
h2 {FONT-FAMILY: Arial, sans-serif; color:#757e47; margin-top:40px; font-size:1.5em}
a.topmenu {text-decoration:none; color: rgb(153, 0, 0); font-size:13px; }
a.topmenu:hover{text-decoration:underline}
p {line-height:120%; }
table {line-height:120%; }
td.tmenu {float:left

Now we'll open iPython in two seperate terminals to demonstrate communication between client and server sockets. See the README for more info.